In [ ]:
# 🟢 Instalar bibliotecas necessárias!pip install pandas openpyxl requests matplotlib seaborn -q# 🟢 Importar bibliotecasimport requestsimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom google.colab import filesfrom datetime import datetime# 🟢 Verificar se a API está onlinedef verificar_api(url):    try:        resposta = requests.get(url, timeout=10)        if resposta.status_code == 200:            print("✅ API do MAPA está online.")            return True        else:            print(f"⚠️ API respondeu com código: {resposta.status_code}")            return False    except Exception as e:        print("❌ Não foi possível conectar à API do MAPA.")        print("Erro:", str(e))        return Falseurl_base = "https://api-agroinsumos.agricultura.gov.br/bioinsumos/v1/produto"params = {"pagina": 1, "quantidade": 100}todos_os_dados = []if verificar_api(url_base):    # 🟢 Requisições paginadas    while True:        response = requests.get(url_base, params=params)        if response.status_code != 200:            print(f"Erro na página {params['pagina']} - Status {response.status_code}")            break        dados = response.json()        resultados = dados.get("listaResultado", [])        if not resultados:            break        for item in resultados:            todos_os_dados.append({                "Nome do Produto": item.get("nomeProduto"),                "Tipo de Uso": item.get("tipoUso"),                "Situação": item.get("situacao"),                "Microrganismos": item.get("principioAtivo", "Não informado"),                "Empresa": item.get("empresa", {}).get("razaoSocial", ""),                "Número de Registro": item.get("numeroRegistro"),                "Detentor do Registro": item.get("detentorRegistro", {}).get("razaoSocial", ""),                "Estado": item.get("empresa", {}).get("estado", ""),                "Cidade": item.get("empresa", {}).get("municipio", ""),                "Contato": item.get("empresa", {}).get("telefone", ""),                "Email": item.get("empresa", {}).get("email", ""),            })        params["pagina"] += 1    # 🟢 Converter em DataFrame    df = pd.DataFrame(todos_os_dados)    hoje = datetime.today().strftime('%d_%m_%Y')    excel_name = f"bioinsumos_{hoje}.xlsx"    csv_name = f"bioinsumos_{hoje}.csv"    df.to_excel(excel_name, index=False)    df.to_csv(csv_name, index=False)    # 🟢 Gráficos    plt.figure(figsize=(10,5))    sns.countplot(data=df, y="Tipo de Uso", order=df["Tipo de Uso"].value_counts().index)    plt.title("Distribuição por Tipo de Uso")    plt.xlabel("Quantidade")    plt.ylabel("Tipo de Uso")    plt.show()    plt.figure(figsize=(10,5))    sns.countplot(data=df, y="Estado", order=df["Estado"].value_counts().index)    plt.title("Distribuição por Estado")    plt.xlabel("Quantidade")    plt.ylabel("Estado")    plt.show()    # 🟢 Downloads    files.download(excel_name)    files.download(csv_name)else:    print("A API está offline. Tente novamente mais tarde.")